# NLLLoss

NLLLoss的全称是Negative Log Likelihood Loss，也就是最大似然函数。

在图片进行单标签分类时，【注意NLLLoss和CrossEntropyLoss都是用于单标签分类，而BCELoss和BECWithLogitsLoss都是使用与多标签分类。这里的多标签是指一个样本对应多个label.】

假设输入m张图片，输出一个m*N的tensor,其中N是分类的个数，比如N为词表大小。比如，输入3张图片，分三类，最后的输出是一个$3*3$的tensor，举一个例子：

In [1]:
import torch
input = torch.randn(3,3)

In [14]:
input

tensor([[ 0.2646, -0.3598, -0.3506],
        [-1.4290, -0.0606,  1.6310],
        [-0.5436, -0.3721,  0.6544]])

假设每一行对应一个样本在3个类别上的输出值，接下来我们可以使用Softmax，来得到每张图片的概率分布：

In [4]:
import torch.nn as nn
softmax = nn.Softmax(dim=1)

In [5]:
softmax_result = softmax(input)

In [6]:
softmax_result

tensor([[0.4817, 0.2580, 0.2604],
        [0.0381, 0.1496, 0.8123],
        [0.1818, 0.2158, 0.6024]])

然后再对softmax的结果取对数：

In [7]:
log_result = torch.log(softmax_result)

In [8]:
log_result

tensor([[-0.7305, -1.3549, -1.3457],
        [-3.2680, -1.8995, -0.2079],
        [-1.7049, -1.5333, -0.5069]])

NLLLoss的结果就是把上面的输出log_result与Label对应的值拿出来，去掉负号再求和取平均。

假设target=[1,0,2]，所以就是取出-1.3549, -3.2680, -0.5069，去掉负号再求和取平均。

具体结果如下：

In [9]:
(1.3549 + 3.2680 + 0.5069)/3

1.709933333333333

下面用NLLLoss来验证一下：

In [12]:
loss_fun = nn.NLLLoss()
target = torch.tensor([1,0,2])
loss = loss_fun(log_result, target)
loss

tensor(1.7099)

可以看到结果是基本一致的.

# CrossEntropyLoss

CrossEntropyLoss就是交叉熵代价函数。

它就是把上面的我们执行的softmax+log+NLLLoss合并起来了，一步执行完。

我们可以来验证一下：

In [13]:
loss_fun2 = nn.CrossEntropyLoss()

In [16]:
input2 = torch.tensor([[ 0.2646, -0.3598, -0.3506],
        [-1.4290, -0.0606,  1.6310],
        [-0.5436, -0.3721,  0.6544]])
target = torch.tensor([1,0,2])


In [18]:
loss_cross = loss_fun2(input2, target)
loss_cross

tensor(1.7099)

可以看到结果是一样的。

总结： NLLLoss和CrossEntropyLoss计算损失函数的形式可以统一为：
$$loss(y_{model},y_{true}) = -\sum y_{true}*log(y_{model})$$

其中$y_{model}$表示模型的输出（经过softmax后的结果）， $y_{true}$表示样本的真实target.

在具体的训练中，其实target是个N维tensor，只有对应的Label位为1，其他的都为0，$y_{model}$也是一个N维的tensor,
这样就实现了上面说的只选取Label位的值参与最后的求和取平均计算。